In [1]:
import re
import pandas as pd
import numpy as np
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
from timeit import default_timer as timer
from datetime import datetime, timedelta
import spacy
from spacy.lang.pl import Polish
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

from collections import Counter

importlib.reload(funcs)

<module 'funcs' from 'C:\\Users\\Mag\\Documents\\_projekty\\praca\\funcs.py'>

In [2]:
sp_joined = pd.read_csv('00_dane/sp_joined.csv',  sep='|', encoding='UTF-8')

In [3]:
len(sp_joined)

158885

#### Usuwam cyfry i znaki interpunkcyjne

In [4]:
%%time
sp_test = sp_joined['speech_rawer']
sp_test = sp_test.apply(lambda x: re.sub('[\W\d]',' ', x.lower()))

#czas dla apply : 1 min 10 s

Wall time: 46.8 s


#### Lista stopwords polskich do usunięcia

In [78]:
nlp = spacy.load('pl_model')
stop = set(nlp.Defaults.stop_words)
stop.add('zł')
stop.add('pkt')
stop.add('art')
stop.add('ustawy')
stop.add('r')

#stop = nlp.Defaults.stop_words

In [79]:
%%time
sp_test = sp_test.apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
#sp_test = [w for w in sp_test if w not in stop]

#Czas dla całości
# 20,9 s


Wall time: 14.2 s


### Ile mamy słów

In [80]:
%%time
corp_dict={}
n = 1
for speech in sp_test:
    if n % 10000 == 0:
        print('Wystąpienie nr: ', n)
    for word in speech.split():
        if word not in corp_dict.keys(): 
            corp_dict[word]=1
        else:
            corp_dict[word]+=1
    n+=1

Wystąpienie nr:  10000
Wystąpienie nr:  20000
Wystąpienie nr:  30000
Wystąpienie nr:  40000
Wystąpienie nr:  50000
Wystąpienie nr:  60000
Wystąpienie nr:  70000
Wystąpienie nr:  80000
Wystąpienie nr:  90000
Wystąpienie nr:  100000
Wystąpienie nr:  110000
Wystąpienie nr:  120000
Wystąpienie nr:  130000
Wystąpienie nr:  140000
Wystąpienie nr:  150000
Wall time: 29.7 s


In [81]:
sp_test.to_csv('00_dane/clear_sp.csv', encoding='UTF=8', sep='|')

In [82]:
len(corp_dict.keys()), sum(corp_dict.values())

(367716, 42115946)

#### Sortowanie słownika po liczbie wystąpień

In [83]:
corp_dict = {k: v for k, v in sorted(corp_dict.items(), key=lambda item: item[1])}

In [84]:
corp_dict['suweren']

207

#### Jaki jest rozkład częstości słów?

Jakie słowa pojawiają się najrzadziej?

In [85]:
u = [k for k,v in corp_dict.items() if v<=5]

In [86]:
len(u)

226599

In [87]:
for _ in range(10):
    print(u[random.randrange(len(u))])

balazsowi
strzybny
kaligraficznym
nieuzgodniona
piruet
wahającym
ukrainki
skserowanie
zapraszającą
skamle


Jakie słowa pojawiają się najczęściej?

In [88]:
sorted(corp_dict.items(), key=lambda item: item[1])[-10:]

[('rząd', 95603),
 ('projektu', 95657),
 ('prawa', 96092),
 ('chodzi', 99480),
 ('panie', 108932),
 ('projekt', 113847),
 ('pytanie', 116217),
 ('pracy', 118301),
 ('komisji', 146153),
 ('państwa', 148339)]

#### Ile słów jest w każdym wystąpieniu?

In [4]:
sp_joined['liczba_sl_raw'] = [len(sp.split()) for sp in sp_joined['speech_raw']]

In [5]:
sp_joined['liczba_sl_rawer'] = [len(sp.split()) for sp in sp_joined['speech_rawer']]

In [8]:
sum(sp_joined['liczba_sl_rawer'])

70671141

In [9]:
sp_joined.to_csv('00_dane/sp_joined.csv', sep='|', encoding='UTF-8')